In [1]:
from bs4 import BeautifulSoup
import requests
import os
from urllib.parse import urljoin
import re
import csv

In [6]:
os.getcwd()

'C:\\Users\\MSI-NB'

In [7]:
# URL of the webpage you want to scrape
url = "https://www.mushroom.world/mushrooms/namelist"  # Replace with the actual URL

# Headers to mimic a real browser visit
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'}

# Send a GET request to the URL
response = requests.get(url, headers=headers)
final_list = []
text_list = []

# Check if the request was successful
if response.status_code == 200:
    # Parse the content of the response with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the parent div with id 'name-list'
    name_list_div = soup.find('div', id='name-list')

    if name_list_div:
        # Find all the divs with class 'item' inside the parent div
        items = name_list_div.find_all('div', class_='item')

        # Iterate through each item to extract the links
        for item in items:
            link = item.find('a')
            if link and link.has_attr('href'):
                #print(link['href'])  # Prints the href attribute and the text of the link
                # URL of the webpage you want to scrape
                url = link['href'] # Assuming you want to scrape the homepage or adjust as needed

                # Send a GET request to the URL
                response = requests.get(url, headers=headers)
                text_result = []
                # Check if the request was successful
                if response.status_code == 200:
                    # Parse the content of the response with BeautifulSoup
                    soup = BeautifulSoup(response.content, 'html.parser')
                    # Extract caption text in parentheses
                    caption_div = soup.find('div', class_='caption')
                    if caption_div:
                        caption_text = caption_div.get_text()
                        text_in_parentheses = re.findall(r'\((.*?)\)', caption_text)
                        text = ', '.join(text_in_parentheses)
                        modified_string = text.lower().replace(" ", "_")
                        text_result.append(modified_string)
                        #print("Common Name:", modified_string)

                    # Extract Labelus and its corresponding Textus or Longtextus contents
                    labels = soup.find_all('div', class_='labelus')

                    for label in labels:
                        # Find the next sibling element that is either 'textus' or 'longtextus'
                        text_element = label.find_next_sibling(class_=['textus', 'longtextus'])

                        if text_element:
                            #print(f"{label.get_text()}: {text_element.get_text()}")
                            text_result.append(text_element.get_text())
                        else:
                            print(f"{label.get_text()}: No corresponding text element found")

                    # Find the parent div with id 'mushroom-list'
                    mushroom_list_div = soup.find('div', id='mushroom-list')

                    if mushroom_list_div:
                        # Find all the divs with class 'image' inside the parent div
                        image_divs = mushroom_list_div.find_all('div', class_='image')

                        # Iterate through each div to extract the image links and names
                        for image_div in image_divs:
                            result = []
                            link = image_div.find('a')
                            if link and link.has_attr('href'):
                                # Construct full image URL
                                image_url = urljoin('https://www.mushroom.world/', link['href'])
                                image_name = os.path.basename(image_url)  # Extracts the name after the last '/'
                                # Extract the value of the 'rel' attribute
                                title = link.get('title')
                                modified_label = title.lower().replace(" ", "_")

                                #print(image_url, image_name, modified_label)   # Prints the full image URL and the image name
                                result.append(modified_string)
                                result.append(modified_label)
                                result.append(image_url)
                                #result.append(image_name)
                                result = result 

                                final_list.append(result)
                        text_result.append(modified_label)
                        #print(final_list)
                        #print(text_result)

                    else:
                        print("No 'mushroom-list' div found on the webpage")
                    text_list.append(text_result)
                else:
                    print("Failed to retrieve the webpage")
        #print(final_list)
        #print(text_result)

    else:
        print("No 'name-list' div found on the webpage")
else:
    print("Failed to retrieve the webpage")


In [8]:
final_list

[['horse_mushroom',
  'agaricus_arvensis',
  'https://www.mushroom.world/data/fungi/Agaricusarvensis1.jpg'],
 ['horse_mushroom',
  'agaricus_arvensis',
  'https://www.mushroom.world/data/fungi/Agaricusarvensis2.jpg'],
 ['horse_mushroom',
  'agaricus_arvensis',
  'https://www.mushroom.world/data/fungi/Agaricusarvensis3.jpg'],
 ['horse_mushroom',
  'agaricus_arvensis',
  'https://www.mushroom.world/data/fungi/Agaricusarvensis4.jpg'],
 ['horse_mushroom',
  'agaricus_arvensis',
  'https://www.mushroom.world/data/fungi/Agaricusarvensis5.jpg'],
 ['horse_mushroom',
  'agaricus_arvensis',
  'https://www.mushroom.world/data/fungi/Agaricusarvensis6.jpg'],
 ['horse_mushroom',
  'agaricus_arvensis',
  'https://www.mushroom.world/data/fungi/Agaricusarvensis7.jpg'],
 ['the_prince',
  'agaricus_augustus',
  'https://www.mushroom.world/data/fungi/Agaricusaugustus1.jpg'],
 ['the_prince',
  'agaricus_augustus',
  'https://www.mushroom.world/data/fungi/Agaricusaugustus2.jpg'],
 ['the_prince',
  'agaricus

In [9]:
text_list

[['horse_mushroom',
  'Agaricaceae',
  'Europe and parts of North America and Asia',
  'Cap 8-20 cm diameter, stem 8-10 cm tall * 1.5-3 cm diameter',
  'This site contains no information about the edibility or toxicity of mushrooms.',
  "Agaricus arvensis, commonly known as the horse mushroom, is a sometimes massive agaric that has a white cap, ringed stem, pink or chocolate-coloured gills and no volva. It grows on soil (often heavily fertilized) or on vegetable debris.\r\n\nCap white or cream-coloured, may have fine scales, first hemispherical in shape before flattening out with maturity. The flesh is white, firm and thick. The cap turns yellowish where bruised, and old caps often take on a yellow-brown tinge. Gills are at first pale pink, becoming pink, then chocolate brown or blackish, free and crowded. Stem is white or cream, slightly club-shaped and smooth or finely scaly below the ring. The ring is white or cream, pendulous and superior. Viewed from below, on a closed-cap specime

In [13]:
# Example data: a list of lists
data = [['Common_Name', 'Scientific_Name', 'Link']] + final_list

# Open a file for writing
with open('mushroom_images.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the data
    for row in data:
        writer.writerow(row)
        
# Example data: a list of lists
data = [['Common_Name', 'Family', 'Location','Dimensions','Edibility','Description','Scientific_Name']] + text_list

# Open a file for writing
with open('mushroom_info.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the data
    for row in data:
        writer.writerow(row)


In [40]:
import os
from urllib.parse import urljoin
import re

# URL of the webpage you want to scrape
url = "https://www.mushroom.world/show?n=Agaricus-arvensis" # Assuming you want to scrape the homepage or adjust as needed

# Send a GET request to the URL
response = requests.get(url, headers=headers)

final_list = []
text_list = []
text_result = []
# Check if the request was successful
if response.status_code == 200:
    # Parse the content of the response with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    # Extract caption text in parentheses
    caption_div = soup.find('div', class_='caption')
    if caption_div:
        caption_text = caption_div.get_text()
        text_in_parentheses = re.findall(r'\((.*?)\)', caption_text)
        text = ', '.join(text_in_parentheses)
        modified_string = text.lower().replace(" ", "_")
        text_result.append(modified_string)
        #print("Common Name:", modified_string)
        
    # Extract Labelus and its corresponding Textus or Longtextus contents
    labels = soup.find_all('div', class_='labelus')

    for label in labels:
        # Find the next sibling element that is either 'textus' or 'longtextus'
        text_element = label.find_next_sibling(class_=['textus', 'longtextus'])

        if text_element:
            #print(f"{label.get_text()}: {text_element.get_text()}")
            text_result.append(text_element.get_text())
        else:
            print(f"{label.get_text()}: No corresponding text element found")

    # Find the parent div with id 'mushroom-list'
    mushroom_list_div = soup.find('div', id='mushroom-list')

    if mushroom_list_div:
        # Find all the divs with class 'image' inside the parent div
        image_divs = mushroom_list_div.find_all('div', class_='image')

        # Iterate through each div to extract the image links and names
        for image_div in image_divs:
            result = []
            link = image_div.find('a')
            if link and link.has_attr('href'):
                # Construct full image URL
                image_url = urljoin('https://www.mushroom.world/', link['href'])
                image_name = os.path.basename(image_url)  # Extracts the name after the last '/'
                # Extract the value of the 'rel' attribute
                title = link.get('title')
                modified_label = title.lower().replace(" ", "_")

                #print(image_url, image_name, modified_label)   # Prints the full image URL and the image name
                result.append(modified_string)
                result.append(modified_label)
                result.append(image_url)
                #result.append(image_name)
                result = result 
                
                final_list.append(result)
        text_result.append(modified_label)
        print(final_list)
        print(text_result)
    
    else:
        print("No 'mushroom-list' div found on the webpage")
else:
    print("Failed to retrieve the webpage")


[['horse_mushroom', 'agaricus_arvensis', 'https://www.mushroom.world/data/fungi/Agaricusarvensis1.jpg'], ['horse_mushroom', 'agaricus_arvensis', 'https://www.mushroom.world/data/fungi/Agaricusarvensis2.jpg'], ['horse_mushroom', 'agaricus_arvensis', 'https://www.mushroom.world/data/fungi/Agaricusarvensis3.jpg'], ['horse_mushroom', 'agaricus_arvensis', 'https://www.mushroom.world/data/fungi/Agaricusarvensis4.jpg'], ['horse_mushroom', 'agaricus_arvensis', 'https://www.mushroom.world/data/fungi/Agaricusarvensis5.jpg'], ['horse_mushroom', 'agaricus_arvensis', 'https://www.mushroom.world/data/fungi/Agaricusarvensis6.jpg'], ['horse_mushroom', 'agaricus_arvensis', 'https://www.mushroom.world/data/fungi/Agaricusarvensis7.jpg']]
['horse_mushroom', 'Agaricaceae', 'Europe and parts of North America and Asia', 'Cap 8-20 cm diameter, stem 8-10 cm tall * 1.5-3 cm diameter', 'This site contains no information about the edibility or toxicity of mushrooms.', "Agaricus arvensis, commonly known as the hor